In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install pandas-datareader

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
pip install pmdarima

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
pip install prophet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
pip install statsmodels

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [19]:
# 월평균기온
weatherData = pd.read_csv("./주요 도시 기후 데이터.csv")
weatherData['시간(UTC)'] = pd.to_datetime(weatherData['시간(UTC)'])

all_predictions = pd.DataFrame(columns=['지점명', '날짜', '월평균 기온'])


filtered_grouped = weatherData.groupby('지점명').filter(lambda x: len(x) >= 20)

for name, group in filtered_grouped.groupby('지점명'):
    group_data = group[['시간(UTC)','월평균 기온']]
    group_data = group_data.set_index('시간(UTC)')

    train_data, test_data = train_test_split(group_data, test_size=0.2, shuffle=False)

    # 최적 파라미터
    model_opt = SARIMAX(train_data['월평균 기온'].values, order=(1, 1, 1), seasonal_order=(0, 1, 1, 12))
    model_opt_fit = model_opt.fit()
    
    # 예측 수행
    future_steps = 100
    future_forecast = model_opt_fit.get_forecast(steps=future_steps)
    future_predicted_value = future_forecast.predicted_mean
    future_predicted_ub = future_forecast.conf_int()[:, 0]
    future_predicted_lb = future_forecast.conf_int()[:, 1]
    future_index = pd.date_range(start=train_data.index[-1] + pd.DateOffset(days=1), periods=future_steps, freq='M')
    df_future = pd.DataFrame({
        '지점명' :name,
        '날짜': future_index.to_period('M'),
        '월평균 기온': future_predicted_value
    })
    
    all_predictions = pd.concat([all_predictions, df_future])
    
all_predictions.reset_index(drop=True, inplace=True)
all_predictions = all_predictions[['지점명', '날짜', '월평균 기온']]
filtered_predictions = all_predictions[(all_predictions['날짜'] >= pd.Period('2023-01')) & 
                                       (all_predictions['날짜'] <= pd.Period('2024-06'))]
# filtered_predictions.to_csv('월평균기온예측.csv', index=False, encoding='utf-8-sig')

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_13884\1512980470.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_predictions = pd.concat([all_predictions, df_future])
c:\Users\SSAFY\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'
c:\Users\SSAFY\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\SSAFY\AppData\Local\Programs\Pytho

In [12]:
# 월평균 일최고기온
weatherData = pd.read_csv("./주요 도시 기후 데이터.csv")
weatherData['시간(UTC)'] = pd.to_datetime(weatherData['시간(UTC)'])
weatherData = weatherData.dropna(subset=['월평균 일최고기온'])

weatherData = weatherData[(weatherData['월평균 일최고기온'] > -20) & (weatherData['월평균 기온'] < 50)]
all_predictions = pd.DataFrame(columns=['지점명', '날짜', '월평균 일최고기온'], dtype=object)

filtered_grouped = weatherData.groupby('지점명').filter(lambda x: len(x) >= 20)

for name, group in filtered_grouped.groupby('지점명'):
    group_data = group[['시간(UTC)','월평균 일최고기온']]
    group_data = group_data.set_index('시간(UTC)')

    train_data, test_data = train_test_split(group_data, test_size=0.2, shuffle=False)

    # 최적 파라미터
    model_opt = SARIMAX(train_data['월평균 일최고기온'].values, order=(1, 1, 1), seasonal_order=(0, 1, 1, 12))
    model_opt_fit = model_opt.fit()
    
    # 예측 수행
    future_steps = 100
    future_forecast = model_opt_fit.get_forecast(steps=future_steps)
    future_predicted_value = future_forecast.predicted_mean
    future_predicted_ub = future_forecast.conf_int()[:, 0]
    future_predicted_lb = future_forecast.conf_int()[:, 1]
    future_index = pd.date_range(start=train_data.index[-1] + pd.DateOffset(days=1), periods=future_steps, freq='M')
    
    df_future = pd.DataFrame({
        '지점명' :name,
        '날짜': future_index.to_period('M'),
        '월평균 일최고기온': future_predicted_value
    })
    
    all_predictions = pd.concat([all_predictions, df_future])
    
all_predictions.reset_index(drop=True, inplace=True)
all_predictions = all_predictions[['지점명', '날짜', '월평균 일최고기온']]
filtered_predictions = all_predictions[(all_predictions['날짜'] >= pd.Period('2023-01')) & 
                                       (all_predictions['날짜'] <= pd.Period('2024-06'))]

# filtered_predictions.to_csv('일최고기온예측.csv', index=False, encoding='utf-8-sig')

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_13884\2088532988.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_predictions = pd.concat([all_predictions, df_future])
c:\Users\SSAFY\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\SSAFY\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'
c:\Users\SSAFY\AppData\Local\Programs\Pytho

In [13]:
# 월평균 일최저기온
weatherData = pd.read_csv("./주요 도시 기후 데이터.csv")
weatherData['시간(UTC)'] = pd.to_datetime(weatherData['시간(UTC)'])
weatherData = weatherData.dropna(subset=['월평균 일최저기온'])

weatherData = weatherData[(weatherData['월평균 일최저기온'] > -40) & (weatherData['월평균 기온'] < 40)]
all_predictions = pd.DataFrame(columns=['지점명', '날짜', '월평균 일최저기온'], dtype=object)

filtered_grouped = weatherData.groupby('지점명').filter(lambda x: len(x) >= 20)

for name, group in filtered_grouped.groupby('지점명'):
    group_data = group[['시간(UTC)','월평균 일최저기온']]
    group_data = group_data.set_index('시간(UTC)')

    train_data, test_data = train_test_split(group_data, test_size=0.2, shuffle=False)

    # 최적 파라미터
    model_opt = SARIMAX(train_data['월평균 일최저기온'].values, order=(1, 1, 1), seasonal_order=(0, 1, 1, 12))
    model_opt_fit = model_opt.fit()
    
    # 예측 수행
    future_steps = 100
    future_forecast = model_opt_fit.get_forecast(steps=future_steps)
    future_predicted_value = future_forecast.predicted_mean
    future_predicted_ub = future_forecast.conf_int()[:, 0]
    future_predicted_lb = future_forecast.conf_int()[:, 1]
    future_index = pd.date_range(start=train_data.index[-1] + pd.DateOffset(days=1), periods=future_steps, freq='M')

    df_future = pd.DataFrame({
        '지점명' :name,
        '날짜': future_index.to_period('M'),
        '월평균 일최저기온': future_predicted_value
    })
    all_predictions = pd.concat([all_predictions, df_future])
    
all_predictions.reset_index(drop=True, inplace=True)
all_predictions = all_predictions[['지점명', '날짜', '월평균 일최저기온']]

filtered_predictions = all_predictions[(all_predictions['날짜'] >= pd.Period('2023-01')) & 
                                       (all_predictions['날짜'] <= pd.Period('2024-06'))]

# filtered_predictions.to_csv('일최저기온예측.csv', index=False, encoding='utf-8-sig')

c:\Users\SSAFY\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\SSAFY\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\SSAFY\AppData\Local\Temp\ipykernel_13884\3829375113.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_predictions = pd.concat([all_predictions, df_future])
c:\Users\SSAFY\AppData\Local\Prog

In [14]:
# 최대 일강수량
weatherData = pd.read_csv("./주요 도시 기후 데이터.csv")
weatherData['시간(UTC)'] = pd.to_datetime(weatherData['시간(UTC)'])
weatherData = weatherData.dropna(subset=['최대 일강수량'])

weatherData = weatherData[weatherData['최대 일강수량'] < 600]
all_predictions = pd.DataFrame(columns=['지점명', '날짜', '최대 일강수량'], dtype=object)

filtered_grouped = weatherData.groupby('지점명').filter(lambda x: len(x) >= 20)

for name, group in filtered_grouped.groupby('지점명'):
    group_data = group[['시간(UTC)','최대 일강수량']]
    group_data = group_data.set_index('시간(UTC)')

    train_data, test_data = train_test_split(group_data, test_size=0.2, shuffle=False)

    # 최적 파라미터
    model_opt = SARIMAX(train_data['최대 일강수량'].values, order=(1, 1, 1), seasonal_order=(0, 1, 1, 12))
    model_opt_fit = model_opt.fit()
    
    # 예측 수행
    future_steps = 100
    future_forecast = model_opt_fit.get_forecast(steps=future_steps)
    future_predicted_value = future_forecast.predicted_mean

    # 음수 값이 나오면 0으로 처리
    future_predicted_value[future_predicted_value < 0] = 0
    future_predicted_ub = future_forecast.conf_int()[:, 0]
    future_predicted_lb = future_forecast.conf_int()[:, 1]
    future_index = pd.date_range(start=train_data.index[-1] + pd.DateOffset(days=1), periods=future_steps, freq='M')

    df_future = pd.DataFrame({
        '지점명' :name,
        '날짜': future_index.to_period('M'),
        '최대 일강수량': future_predicted_value
    })
    all_predictions = pd.concat([all_predictions, df_future])
    
all_predictions.reset_index(drop=True, inplace=True)
all_predictions = all_predictions[['지점명', '날짜', '최대 일강수량']]
filtered_predictions = all_predictions[(all_predictions['날짜'] >= pd.Period('2023-01')) & (all_predictions['날짜'] <= pd.Period('2024-06'))]

# filtered_predictions.to_csv('최대일강수량예측.csv', index=False, encoding='utf-8-sig')

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_13884\236494090.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_predictions = pd.concat([all_predictions, df_future])
c:\Users\SSAFY\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'
c:\Users\SSAFY\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'
c:\Users\SSAFY\AppData\Local\Programs

In [15]:
# 최대 일강수량
weatherData = pd.read_csv("./주요 도시 기후 데이터.csv")
weatherData['시간(UTC)'] = pd.to_datetime(weatherData['시간(UTC)'])
weatherData = weatherData.dropna(subset=['뇌우일수'])

all_predictions = pd.DataFrame(columns=['지점명', '날짜', '뇌우일수'], dtype=object)

filtered_grouped = weatherData.groupby('지점명').filter(lambda x: len(x) >= 20)

for name, group in filtered_grouped.groupby('지점명'):
    group_data = group[['시간(UTC)','뇌우일수']]
    group_data = group_data.set_index('시간(UTC)')

    train_data, test_data = train_test_split(group_data, test_size=0.2, shuffle=False)

    # 최적 파라미터
    model_opt = SARIMAX(train_data['뇌우일수'].values, order=(1, 1, 1), seasonal_order=(0, 1, 1, 12))
    model_opt_fit = model_opt.fit()
    
    # 예측 수행
    future_steps = 100
    future_forecast = model_opt_fit.get_forecast(steps=future_steps)
    future_predicted_value = future_forecast.predicted_mean

    # 음수 값이 나오면 0으로 처리
    future_predicted_value[future_predicted_value < 0] = 0
    future_predicted_ub = future_forecast.conf_int()[:, 0]
    future_predicted_lb = future_forecast.conf_int()[:, 1]
    future_index = pd.date_range(start=train_data.index[-1] + pd.DateOffset(days=1), periods=future_steps, freq='M')

    df_future = pd.DataFrame({
        '지점명' :name,
        '날짜': future_index.to_period('M'),
        '뇌우일수': future_predicted_value
    })
    all_predictions = pd.concat([all_predictions, df_future])
    
all_predictions.reset_index(drop=True, inplace=True)
all_predictions = all_predictions[['지점명', '날짜', '뇌우일수']]
filtered_predictions = all_predictions[(all_predictions['날짜'] >= pd.Period('2023-01')) & (all_predictions['날짜'] <= pd.Period('2024-06'))]

# filtered_predictions.to_csv('뇌우일수예측.csv', index=False, encoding='utf-8-sig')

c:\Users\SSAFY\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\SSAFY\AppData\Local\Temp\ipykernel_13884\974813971.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_predictions = pd.concat([all_predictions, df_future])
c:\Users\SSAFY\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parame